In [20]:
import pandas as pd

In [30]:
obs = pd.read_csv('./data/beetle/artportalen_final.csv')
obs["Month"] = pd.to_datetime(obs['Date']).dt.to_period("M").dt.to_timestamp()
print(f"Datapoints (observations): {len(obs)}")
obs.head(5)

Datapoints (observations): 1022


,Lat,Lon,Kommun,Lan,Quantity,Date,Month
0,64.024023,20.650910,Robertsfors,Västerbotten,1,2018-09-14 00:00:00,2018-09-01
1,56.729677,15.956413,Nybro,Kalmar,1,2018-09-14 00:00:00,2018-09-01
2,55.614954,14.276141,Simrishamn,Skåne,10,2018-09-16 00:00:00,2018-09-01
3,61.714395,17.372628,Hudiksvall,Gävleborg,1,2018-06-25 00:00:00,2018-06-01
4,56.730931,15.906116,Nybro,Kalmar,1,2018-09-19 00:00:00,2018-09-01


In [43]:
try:
    weather_features = pd.read_csv('./data/weather_monthly/weather_final.csv')
    weather_features['Month'] = pd.to_datetime(weather_features['Month'])
    print(f"Datapoints (weather): {len(weather_features)}")
except:
    pass # TODO: RUN data_weather.ipynb

try:
    ndvi_features = pd.read_csv('./data/ndvi/ndvi_final.csv')
    ndvi_features['Month'] = pd.to_datetime(ndvi_features['Month'])
    print(f"Datapoints (NDVI): {len(ndvi_features)}")
except:
    pass # TODO: RUN data_ndvi.ipynb


Datapoints (weather): 992
Datapoints (NDVI): 1022


In [36]:
weather_features.head(5)

,Lat,Lon,time,t2m,tp,swvl1,swvl2,ssrd
0,64.024023,20.650910,2018-09-01,284.20142,0.001569,0.215851,0.208481,9420792.0
1,56.729677,15.956413,2018-09-01,287.38500,0.000978,0.212662,0.211121,10295400.0
2,61.714395,17.372628,2018-06-01,287.19800,0.001399,0.103455,0.144211,22301380.0
3,56.730931,15.906116,2018-09-01,287.15454,0.001042,0.216354,0.214478,10131648.0
4,64.025160,20.649693,2018-09-01,284.09204,0.001598,0.217621,0.210419,9403106.0


In [40]:
ndvi_features.head(5)

,Lat,Lon,Month,NDVI,NDVI_clim,NDVI_anom,NDVI_lag1,NDVI_anom_lag1,NDVI_lag2,NDVI_anom_lag2
0,64.024023,20.650910,2018-09-01,0.60930,0.623756,-0.014456,0.61225,-0.076781,0.65995,-0.043456
1,56.729677,15.956413,2018-09-01,0.60710,0.600225,0.006875,0.61515,0.038256,0.61625,-0.026462
2,55.614954,14.276141,2018-09-01,0.72935,0.694438,0.034912,0.69880,0.007531,0.67365,-0.029069
3,61.714395,17.372628,2018-06-01,0.74255,0.774188,-0.031637,0.70265,-0.004019,0.41235,-0.223112
4,56.730931,15.906116,2018-09-01,0.69140,0.724681,-0.033281,0.70405,-0.047056,0.69195,-0.057462


In [ ]:
X = obs.merge(ndvi_features, on=["Lat", "Lon", "Month"], how="left")
X = X.merge(weather_features, on=["Lat", "Lon", "Month"], how="left")
X.dropna(axis=0,inplace=True)
X

,Lat,Lon,Kommun,Lan,Quantity,Date,Month,NDVI,NDVI_clim,NDVI_anom,NDVI_lag1,NDVI_anom_lag1,NDVI_lag2,NDVI_anom_lag2
0,64.024023,20.650910,Robertsfors,Västerbotten,1,2018-09-14 00:00:00,2018-09-01,0.60930,0.623756,-0.014456,0.61225,-0.076781,0.65995,-0.043456
1,56.729677,15.956413,Nybro,Kalmar,1,2018-09-14 00:00:00,2018-09-01,0.60710,0.600225,0.006875,0.61515,0.038256,0.61625,-0.026462
2,55.614954,14.276141,Simrishamn,Skåne,10,2018-09-16 00:00:00,2018-09-01,0.72935,0.694438,0.034912,0.69880,0.007531,0.67365,-0.029069
3,61.714395,17.372628,Hudiksvall,Gävleborg,1,2018-06-25 00:00:00,2018-06-01,0.74255,0.774188,-0.031637,0.70265,-0.004019,0.41235,-0.223112
4,56.730931,15.906116,Nybro,Kalmar,1,2018-09-19 00:00:00,2018-09-01,0.69140,0.724681,-0.033281,0.70405,-0.047056,0.69195,-0.057462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,61.006044,15.190126,Rättvik,Dalarna,1,2024-10-26 00:00:00,2024-10-01,0.68830,0.755243,-0.066943,0.78480,0.046519,0.76730,0.024381
1080,57.858032,15.090049,Ydre,Östergötland,1,2025-04-30 00:00:00,2025-04-01,0.65635,0.671231,-0.014881,0.59145,-0.006969,0.61105,0.069121
1081,59.993022,18.877981,Norrtälje,Stockholm,5,2025-11-25 00:00:00,2025-11-01,0.31505,0.376812,-0.061762,0.63320,0.056314,0.78280,0.038856
1082,58.913920,14.528770,Laxå,Örebro,1,2024-09-05 00:00:00,2024-09-01,0.77580,0.783181,-0.007381,0.78690,0.010412,0.81225,0.015844
